In [ ]:
activation=nn.Tanh

class pinn(nn.Module):
    def __init__(self, hidden_size=16,hidden_layers=4):
        super(pinn, self).__init__()
        layers= [nn.Linear(2, hidden_size), activation()]
        for _ in range(hidden_layers):
            layers.append(nn.Linear(hidden_size, hidden_size))
            layers.append(activation())
        layers.append(nn.Linear(hidden_size, 1))
        self.network = nn.Sequential(*layers)
    def forward(self, x):
        return self.network(x)

#Defining changeable parameters:
epochs=1000
num_training_points=1000

model=pinn(hidden_size=20, hidden_layers=3) # to be used for evaluating u at boundaries and inside the domain

# Changable parameters:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001) #Using the Adam optimizer
mse_loss = nn.MSELoss() #Using the Mean Squared Error loss function

#Defining PDE
def laplace_residual(u, x, y):
    # Compute first derivatives with respect to x and y
    du_dx = torch.autograd.grad(u, x, grad_outputs=torch.ones_like(u), create_graph=True)[0]
    du_dy = torch.autograd.grad(u, y, grad_outputs=torch.ones_like(u), create_graph=True)[0]
    
    # Compute second derivatives
    d2u_dx2 = torch.autograd.grad(du_dx, x, grad_outputs=torch.ones_like(du_dx), create_graph=True)[0]
    d2u_dy2 = torch.autograd.grad(du_dy, y, grad_outputs=torch.ones_like(du_dy), create_graph=True)[0]
    
    # Laplace residual
    laplace_residual = torch.mean((d2u_dx2 + d2u_dy2)**2)
    return laplace_residual

#_______________________________________________________________________________________
#Vertical boundary condition A, vertical_a (v_a) paramteres:
v_a_x=0   #x-coordinate of the vertical boundary
v_a_num_points=100  #Number of points on the vertical boundary
v_a_head=0  #Dirichtlet value of the boundary condition
v_a_y_start=0   #Starting y-coordinate of the vertical boundary
v_a_y_end=1     #Ending y-coordinate of the vertical boundary

#Generating tensors for the vertical boundary condition A
v_a_x_points=torch.full((v_a_num_points,1),v_a_x)
v_a_y_points = torch.linspace(v_a_y_start, v_a_y_end, v_a_num_points).reshape(-1,1) #Should maybe change this to rand()?
v_a_boundary_points=torch.cat((v_a_x_points,v_a_y_points),dim=1)
v_a_boundary_target= torch.full((v_a_num_points,1),v_a_head)  #This may have to be in a list?

#Vertical boundary condition B, vertical_b (v_b) paramteres:
v_b_x=1   #x-coordinate of the vertical boundary
v_b_num_points=100  #Number of points on the vertical boundary
v_b_head=1  #Dirichtlet value of the boundary condition
v_b_y_start=0   #Starting y-coordinate of the vertical boundary
v_b_y_end=1     #Ending y-coordinate of the vertical boundary

#Generating tensors for the vertical boundary condition B
v_b_x_points=torch.full((v_b_num_points,1),v_b_x)
v_b_y_points = torch.linspace(v_b_y_start, v_b_y_end, v_b_num_points).reshape(-1,1) #Should maybe change this to rand()?
v_b_boundary_points=torch.cat((v_b_x_points,v_b_y_points),dim=1)
v_b_boundary_target= torch.full((v_b_num_points,1),v_b_head)  #This may have to be in a list?

#Horizontal boundary condition A, horizontal_a (h_a) paramteres:


#Boundary conditions:
def loss_vertical_boundary(model,type,boundary_points,boundary_target):
    if type=="dirichlet":
        u_pred = model(boundary_points)
        # boundary_target= torch.full((num_boundary_points,1),head)  ##could be torch and inside for dynamic boundary points.
        boundary_residual= u_pred-boundary_target
        return torch.mean((boundary_residual)**2)
    elif type=="neumann":

        return 'nan'
    else:
        raise ValueError("Invalid boundary condition type")


# Creating domain points
x = torch.rand((num_training_points,1),requires_grad=True) #Need to be changed to the actual domain
y = torch.rand((num_training_points,1),requires_grad=True) #Need to be changed to the actual domain
train_points=torch.cat([x,y],1) 

#Training loop:
for epoch in range(epochs):
    optimizer.zero_grad() #Have to zero the gradients at the start of each epoch
    
    u_pred=model(train_points)
    
    #losses
    loss_laplace=laplace_residual(u_pred,x,y)
    #Boundary condition loss
    loss_boundary_a=loss_vertical_boundary(model,"dirichlet",v_a_boundary_points,v_a_boundary_target)
    loss_boundary_b=loss_vertical_boundary(model,"dirichlet",v_b_boundary_points,v_b_boundary_target)

    loss=loss_laplace + loss_boundary_a+loss_boundary_b
    loss.backward()
    optimizer.step()

In [ ]:
# Define the neural network for the Physics-Informed Neural Network (PINN)
class PINN(nn.Module):
    def __init__(self, hidden_dim=20, num_hidden_layers=3):
        super(PINN, self).__init__()
        layers = [nn.Linear(2, hidden_dim), nn.Tanh()]
        for _ in range(num_hidden_layers - 1):
            layers.append(nn.Linear(hidden_dim, hidden_dim))
            layers.append(nn.Tanh())
        layers.append(nn.Linear(hidden_dim, 1))
        self.network = nn.Sequential(*layers)
        
    def forward(self, x):
        return self.network(x)

# Physics-Informed Loss Function
def laplace_loss(model, coords):
    u = model(coords)
    grads = torch.autograd.grad(u, coords, grad_outputs=torch.ones_like(u), create_graph=True)[0]
    u_xx = torch.autograd.grad(grads[:, 0], coords, grad_outputs=torch.ones_like(grads[:, 0]), create_graph=True)[0][:, 0]
    u_yy = torch.autograd.grad(grads[:, 1], coords, grad_outputs=torch.ones_like(grads[:, 1]), create_graph=True)[0][:, 1]
    return torch.mean((u_xx + u_yy) ** 2)

# Boundary conditions
def boundary_loss(model, coords, target):
    u_pred = model(coords)
    return torch.mean((u_pred - target) ** 2)

# Generate coordinates and boundary conditions
def generate_data(num_domain_points=1000, num_boundary_points=100):
    # Domain points
    x = torch.rand((num_domain_points, 1), requires_grad=True)
    y = torch.rand((num_domain_points, 1), requires_grad=True)
    domain_points = torch.cat([x, y], dim=1)

    # Boundary points and target values
    boundary_points = []
    target_values = []

    # Boundary conditions for a square domain: [0, 1] x [0, 1]
    
    # u(x, 0) = x and u(x, 1) = x
    x_boundary = torch.rand((num_boundary_points, 1))
    boundary_points.append(torch.cat([x_boundary, torch.zeros((num_boundary_points, 1))], dim=1))  # y = 0
    target_values.append(x_boundary)
    boundary_points.append(torch.cat([x_boundary, torch.ones((num_boundary_points, 1))], dim=1))   # y = 1
    target_values.append(x_boundary)

    # u(0, y) = 0
    y_boundary = torch.rand((num_boundary_points, 1))
    boundary_points.append(torch.cat([torch.zeros((num_boundary_points, 1)), y_boundary], dim=1))
    target_values.append(torch.zeros((num_boundary_points, 1)))

    # u(1, y) = 1
    boundary_points.append(torch.cat([torch.ones((num_boundary_points, 1)), y_boundary], dim=1))
    target_values.append(torch.ones((num_boundary_points, 1)))

    boundary_points = torch.cat(boundary_points, dim=0).requires_grad_(True)
    target_values = torch.cat(target_values, dim=0)

    return domain_points, boundary_points, target_values

# Initialize model, optimizer, and training loop
model = PINN(hidden_dim=20, num_hidden_layers=3)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Generate data
domain_points, boundary_points, boundary_targets = generate_data()

# Training loop
epochs = 1000
for epoch in range(epochs):
    optimizer.zero_grad()
    
    # Compute losses
    domain_loss = laplace_loss(model, domain_points)
    boundary_loss_val = boundary_loss(model, boundary_points, boundary_targets)
    
    # Total loss
    loss = domain_loss + boundary_loss_val
    loss.backward()
    optimizer.step()

    # Logging
    if epoch % 500 == 0:
        print(f"Epoch {epoch}, Total Loss: {loss.item()}, Domain Loss: {domain_loss.item()}, Boundary Loss: {boundary_loss_val.item()}")
